In [2]:
import pandas as pd
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
df = pd.read_csv('twitter_dataset.csv') 
df["Text"] = df["Text"].astype(str)
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format="%Y-%m-%d %H:%M:%S")
df['Timestamp'] = df['Timestamp'].dt.strftime("%d-%m-%Y %H:%M:%S")
df = df.drop(columns=["Username", "Retweets", "Likes"])

In [13]:
# devide the dataframe in 10 
cutting_size = len(df)//10


In [24]:
df1 = df.iloc[: cutting_size]

In [27]:
def sentiment_analysis_transformers(df):
    
    model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual"
    analyser = pipeline("sentiment-analysis", model=model_path)
    df['scores'] = df['Text'].apply(lambda text: analyser (text.lower()))
    df['Sentiment'] = df['scores'].apply(lambda output: output[0]['label']) 
    df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'positive' else (-1 if x == 'negative' else 0)) 
    
    # df['Sentiment'] = df['Sentiment'].apply(lambda x: 'Positive' if x == 'positive' else ('Negative' if x == 'negative' else 'Neutral')) 
    
    # df['Sentiment'] = df['Sentiment'].apply(lambda x: 'Positive ❤️' if x == 'positive' else ('Negative 😡' if x == 'negative' else 'Neutral 💛')) 
    
    df =  df.drop(columns = ["scores"])
    return df

In [17]:
len(df1)

1000

In [ ]:
transformers_result = sentiment_analysis_transformers(df1)

In [21]:
transformers_result

,Tweet_ID,Text,Timestamp,Sentiment
0,1,Party least receive say or single. Prevent pre...,30-01-2023 11:00:51,Positive
1,2,Hotel still Congress may member staff. Media d...,02-01-2023 22:45:58,Neutral
2,3,Nice be her debate industry that year. Film wh...,18-01-2023 11:25:19,Positive
3,4,Laugh explain situation career occur serious. ...,10-04-2023 22:06:29,Neutral
4,5,Involve sense former often approach government...,24-01-2023 07:12:21,Neutral
...,...,...,...,...
995,996,Make environment actually feeling hit need. Be...,29-01-2023 05:36:17,Neutral
996,997,Just him investment state her either trial. Fo...,20-01-2023 13:10:51,Neutral
997,998,Similar also finish analysis.\nIndividual bene...,12-04-2023 09:51:11,Neutral
998,999,Possible resource early itself head federal. T...,28-03-2023 03:02:53,Neutral


In [39]:
df2 = pd.read_csv('Twitter_Data.csv') 
df2["clean_text"] = df2["clean_text"].astype(str)
df2

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [40]:
test_data = df2.iloc[:cutting_size]

In [41]:
len(test_data)

1000

In [42]:
def sentiment_analysis1(df):
    
    model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual"
    analyser = pipeline("sentiment-analysis", model=model_path)
    df['scores'] = df['clean_text'].apply(lambda text: analyser (text.lower()))
    df['Sentiment'] = df['scores'].apply(lambda output: output[0]['label']) 
    df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'positive' else (-1 if x == 'negative' else 0)) 
    
    df =  df.drop(columns = ["scores"])
    return df

In [43]:
test_result = sentiment_analysis1(test_data)

d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\beaut\AppData\Local\Temp\ipykernel_24748\1942290126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['scores'] = df['clean_text'].apply(lambda text: analyser (text.lower()))
C:\Users\beaut\AppData\Local\Temp\ipykernel_24748\1942290126.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [49]:
from sklearn.metrics import confusion_matrix,classification_report

# Calculate the confusion matrix
conf_matrix = confusion_matrix(test_result["category"], test_result["Sentiment"])

# Calculate the accuracy
accuracy = classification_report(test_result["category"], test_result["Sentiment"])

print("Confusion Matrix:")
print(conf_matrix)
print("\nAccuracy:", accuracy)

Confusion Matrix:
[[170  38  17]
 [125 130  79]
 [182  93 166]]

Accuracy:               precision    recall  f1-score   support

        -1.0       0.36      0.76      0.48       225
         0.0       0.50      0.39      0.44       334
         1.0       0.63      0.38      0.47       441

    accuracy                           0.47      1000
   macro avg       0.50      0.51      0.46      1000
weighted avg       0.53      0.47      0.46      1000



In [51]:
# Load your CSV file
data = test_data

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to analyze sentiment and classify based on compound score
def classify_sentiment(text):
    # Get sentiment scores
    scores = analyzer.polarity_scores(text)
    
    # Get individual scores
    neg_score = scores['neg']
    neu_score = scores['neu']
    pos_score = scores['pos']
    compound_score = scores['compound']
    
    # Check if compound score is the highest
    if compound_score >= max(neg_score, neu_score, pos_score):
        # Classify based on compound score
        if compound_score >= 0.05:
            return 1
        elif compound_score > -0.05:
            return 0
        else:
            return -1
    else:
        # Classify based on the highest individual score
        highest_score = max(neg_score, neu_score, pos_score)
        if highest_score == neg_score:
            return -1
        elif highest_score == neu_score:
            return 0
        else:
            return 1
text = "This movie was okay, not great but not terrible"
print(classify_sentiment(text))

# # Apply the sentiment analysis function to the 'text' column
data['Sentiment'] = data['clean_text'].apply(classify_sentiment)
data = data.drop(columns=["scores"])
# data['Sentiment'] = data['Text'].apply(classify_sentiment)
data
# Save the results to a new CSV file
# data.to_csv('data_with_sentiment.csv', index=False)




0


C:\Users\beaut\AppData\Local\Temp\ipykernel_24748\767759645.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sentiment'] = data['clean_text'].apply(classify_sentiment)


,clean_text,category,Sentiment
0,when modi promised “minimum government maximum...,-1.0,0
1,talk all the nonsense and continue all the dra...,0.0,0
2,what did just say vote for modi welcome bjp t...,1.0,0
3,asking his supporters prefix chowkidar their n...,1.0,0
4,answer who among these the most powerful world...,1.0,0
...,...,...,...
995,another meltdown india which will further rejo...,1.0,0
996,there are two reasons for atmosphere hatred cr...,0.0,0
997,modi has wiped out the small micro industries ...,-1.0,0
998,bjp struggles find candidates west bengal graf...,-1.0,0


In [52]:
from sklearn.metrics import confusion_matrix,classification_report

# Calculate the confusion matrix
conf_matrix = confusion_matrix(data["category"], data["Sentiment"])

# Calculate the accuracy
accuracy = classification_report(data["category"], data["Sentiment"])

print("Confusion Matrix:")
print(conf_matrix)
print("\nAccuracy:", accuracy)

Confusion Matrix:
[[  9 201  15]
 [  2 309  23]
 [  1 341  99]]

Accuracy:               precision    recall  f1-score   support

        -1.0       0.75      0.04      0.08       225
         0.0       0.36      0.93      0.52       334
         1.0       0.72      0.22      0.34       441

    accuracy                           0.42      1000
   macro avg       0.61      0.40      0.31      1000
weighted avg       0.61      0.42      0.34      1000



In [54]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load CSV file
# data = pd.read_csv('data.csv')
data_gpt  = test_data.loc[:100]

# Function to analyze sentiment
def analyze_sentiment(text):
    prompt = f"What is the sentiment of the following text? '{text}'"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=100, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Apply sentiment analysis to each row in the 'text' column
data_gpt['Sentiment'] = data['clean_text'].apply(analyze_sentiment)

# # Save the results to a new CSV file
# data.to_csv('data_with_sentiment.csv', index=False)

# print("Sentiment analysis complete! Results saved to 'data_with_sentiment.csv'.")
# print(analyze_sentiment(text))
data_gpt


d:\learning\LIVE-TWITTER-SENTIMENT-ANALYSIS\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting

ValueError: Input length of input_ids is 102, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

# Calculate the confusion matrix
conf_matrix = confusion_matrix(data_gpt["category"], data_gpt["Sentiment"])

# Calculate the accuracy
accuracy = classification_report(data_gpt["category"], data_gpt["Sentiment"])

print("Confusion Matrix:")
print(conf_matrix)
print("\nAccuracy:", accuracy)

In [ ]:
# hashtag gen
# topic classififcation
# data cleaning and enreichement
# define the terms of the projet ie
# sentiment anal, ETL, pipeline, real time, batch,diragrams, usecase, class, package,sequence